# Changing Music Preferences
### Do Spotify users listen to different music during the COVID-19 pandemic? 
### Does this relate to the amount of confirmed cases?

To Do: 

1. Scraping Spotify Top 200 data
2. Get song metadata through Spotify API
3. Data Visualization & Modelling
4. Give reference values for happy/sad music
5. Create interactive dashboard in Shiny

cool plot: https://towardsdatascience.com/discovering-your-music-taste-with-python-and-spotify-api-b51b0d2744d 


## 1.
## Scraping the Top 200 data for 2019 (baseline) and 2020 (pandemic).

https://spotifycharts.com contains the official Spotify Charts. 

Luckily, each day/week is available as a .csv file!  
I will download each file and add them together in one dataframe.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import io

In [2]:
# Scraping all the links to the official Spotify Top 200 .csv's
url = 'https://spotifycharts.com/regional/nl/weekly/latest'

r = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [3]:
dates = [item["data-value"] for item in soup.find_all(attrs={"data-value": True}) if item["data-value"].startswith('2')]
dates[:5]

['2020-12-18--2020-12-25',
 '2020-12-11--2020-12-18',
 '2020-12-04--2020-12-11',
 '2020-11-27--2020-12-04',
 '2020-11-20--2020-11-27']

In [4]:
urls = ["https://spotifycharts.com/regional/nl/weekly/" + date + "/download" for date in dates]
urls[:5]

['https://spotifycharts.com/regional/nl/weekly/2020-12-18--2020-12-25/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-12-11--2020-12-18/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-12-04--2020-12-11/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-11-27--2020-12-04/download',
 'https://spotifycharts.com/regional/nl/weekly/2020-11-20--2020-11-27/download']

## Attention: running the block below will result in downloading 200 .csv's!

In [5]:
data = []

for date in tqdm(dates): 
    url = "https://spotifycharts.com/regional/nl/weekly/" + date + "/download"
    response = requests.get(url)
    file_object = io.StringIO(response.content.decode('utf-8'))
    df = pd.read_csv(file_object, header=1)
    df["Date"] = date
    data.append(df)

# merge all csv's
df = pd.concat(data)
df.reset_index(drop=True, inplace=True)
df.to_csv("..\\data\\raw\\top200_2017_2020.csv", index=False)
data = []

100%|██████████| 209/209 [04:51<00:00,  1.40s/it]


In [6]:
df = pd.read_csv("..\\data\\raw\\top200_2017_2020.csv")
df

,Position,Track Name,Artist,Streams,URL,Date
0,1,All I Want for Christmas Is You,Mariah Carey,1929800,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2020-12-18--2020-12-25
1,2,Last Christmas,Wham!,1845748,https://open.spotify.com/track/2FRnf9qhLbvw8fu...,2020-12-18--2020-12-25
2,3,Santa Tell Me,Ariana Grande,1501707,https://open.spotify.com/track/0lizgQ7Qw35od7C...,2020-12-18--2020-12-25
3,4,It's Beginning to Look a Lot like Christmas,Michael Bublé,1437635,https://open.spotify.com/track/0lLdorYw7lVrJyd...,2020-12-18--2020-12-25
4,5,It's the Most Wonderful Time of the Year,Andy Williams,1386498,https://open.spotify.com/track/1IcR6RlgvDczfvo...,2020-12-18--2020-12-25
...,...,...,...,...,...,...
41795,196,Sex,Cheat Codes,114030,https://open.spotify.com/track/5DA77EqppDmCTWG...,2016-12-23--2016-12-30
41796,197,Ain't My Fault,Zara Larsson,113974,https://open.spotify.com/track/0ADG9OgdVTL7fgR...,2016-12-23--2016-12-30
41797,198,Please Come Home for Christmas,Luther Vandross,113779,https://open.spotify.com/track/2mOtx6P21hecOcP...,2016-12-23--2016-12-30
41798,199,Jodge Me Niet - Titelsong Van De Film “SOOF 2”,Jayh,113763,https://open.spotify.com/track/2VxAfqI3vIOaPSl...,2016-12-23--2016-12-30


In [7]:
print(df.isna().sum())
df.dropna(inplace=True) # removing 8 rows that contain a missing value

df[['Start Week', 'End Week']] = df['Date'].str.split('--', 1, expand=True) # add columns for start and end dates
df['ID'] = df['URL'].str.rsplit('https://open.spotify.com/track/', 1, expand=True)[1] # add ID column for merge in task 2

df.to_csv("..\\data\\processed\\top200_2017_2020.csv", index=False)

Position      0
Track Name    7
Artist        7
Streams       0
URL           2
Date          0
dtype: int64


In [8]:
df = pd.read_csv("..\\data\\processed\\top200_2017_2020.csv")
df.head()

,Position,Track Name,Artist,Streams,URL,Date,Start Week,End Week,ID
0,1,All I Want for Christmas Is You,Mariah Carey,1929800,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2020-12-18--2020-12-25,2020-12-18,2020-12-25,0bYg9bo50gSsH3LtXe2SQn
1,2,Last Christmas,Wham!,1845748,https://open.spotify.com/track/2FRnf9qhLbvw8fu...,2020-12-18--2020-12-25,2020-12-18,2020-12-25,2FRnf9qhLbvw8fu4IBXx78
2,3,Santa Tell Me,Ariana Grande,1501707,https://open.spotify.com/track/0lizgQ7Qw35od7C...,2020-12-18--2020-12-25,2020-12-18,2020-12-25,0lizgQ7Qw35od7CYaoMBZb
3,4,It's Beginning to Look a Lot like Christmas,Michael Bublé,1437635,https://open.spotify.com/track/0lLdorYw7lVrJyd...,2020-12-18--2020-12-25,2020-12-18,2020-12-25,0lLdorYw7lVrJydTINhWdI
4,5,It's the Most Wonderful Time of the Year,Andy Williams,1386498,https://open.spotify.com/track/1IcR6RlgvDczfvo...,2020-12-18--2020-12-25,2020-12-18,2020-12-25,1IcR6RlgvDczfvoWJSSY2A


## 2.
## Retrieve and Combine music features through Spotify API
Spotify generates features on all their music, which can be accessed through the API.  
For each song (ID) in the Top 200 dataset, I will requrest these features and add it to the dataset.

More on the features: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

In [9]:
# Fill this in using your Spotify API Credentials from https://developer.spotify.com/dashboard/applications
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="XXX" 
secret = "XXX"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [10]:
# Where I save my Spotify API Credentials, outside of this public repository :)
import spotipy
%run credentials.py

In [11]:
df_ids = df["ID"].drop_duplicates()     # store all unique ID's in a seperate df

rows = []
batchsize = 100     # maximum batchsize is 100

for i in tqdm(range(0, len(df_ids), batchsize)):
    batch = df_ids[i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            continue
        else:
            rows.append(t)

100%|██████████| 53/53 [00:09<00:00,  5.39it/s]


In [12]:
features = pd.DataFrame.from_dict(rows, orient="columns")
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.336,0.627,7,-7.463,1,0.0384,0.1640,0.000000,0.0708,0.350,150.273,audio_features,0bYg9bo50gSsH3LtXe2SQn,spotify:track:0bYg9bo50gSsH3LtXe2SQn,https://api.spotify.com/v1/tracks/0bYg9bo50gSs...,https://api.spotify.com/v1/audio-analysis/0bYg...,241107,4
1,0.735,0.478,2,-12.472,1,0.0293,0.1890,0.000002,0.3550,0.947,107.682,audio_features,2FRnf9qhLbvw8fu4IBXx78,spotify:track:2FRnf9qhLbvw8fu4IBXx78,https://api.spotify.com/v1/tracks/2FRnf9qhLbvw...,https://api.spotify.com/v1/audio-analysis/2FRn...,262960,4
2,0.525,0.621,7,-7.364,1,0.1160,0.0489,0.000000,0.2940,0.591,191.900,audio_features,0lizgQ7Qw35od7CYaoMBZb,spotify:track:0lizgQ7Qw35od7CYaoMBZb,https://api.spotify.com/v1/tracks/0lizgQ7Qw35o...,https://api.spotify.com/v1/audio-analysis/0liz...,204093,4
3,0.339,0.214,4,-11.714,1,0.0375,0.9080,0.000007,0.3410,0.363,94.775,audio_features,0lLdorYw7lVrJydTINhWdI,spotify:track:0lLdorYw7lVrJydTINhWdI,https://api.spotify.com/v1/tracks/0lLdorYw7lVr...,https://api.spotify.com/v1/audio-analysis/0lLd...,206640,3
4,0.234,0.526,2,-11.656,1,0.0411,0.8710,0.000002,0.1220,0.649,201.739,audio_features,1IcR6RlgvDczfvoWJSSY2A,spotify:track:1IcR6RlgvDczfvoWJSSY2A,https://api.spotify.com/v1/tracks/1IcR6RlgvDcz...,https://api.spotify.com/v1/audio-analysis/1IcR...,151000,3


In [13]:
# drop redundant features
features.drop(['analysis_url','track_href','type','uri'], axis=1, inplace=True) 

In [14]:
features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,5270.000000,5270.000000,5270.000000,5270.000000,5270.000000,5270.000000,5270.000000,5270.000000,5270.000000,5270.000000,5270.000000,5.270000e+03,5270.000000
mean,0.703378,0.639836,5.313472,-6.735502,0.520683,0.166278,0.231481,0.008979,0.167309,0.522509,119.227067,1.911992e+05,3.990512
std,0.137754,0.153018,3.630237,2.353961,0.499619,0.131550,0.227107,0.062384,0.124101,0.211771,28.995551,4.393340e+04,0.328192
min,0.153000,0.031600,0.000000,-23.023000,0.000000,0.023200,0.000055,0.000000,0.018300,0.036000,38.796000,3.013300e+04,1.000000
25%,0.615000,0.550000,2.000000,-7.807750,0.000000,0.054000,0.054925,0.000000,0.093825,0.361000,97.294500,1.642818e+05,4.000000
50%,0.721500,0.649000,6.000000,-6.410500,1.000000,0.120000,0.153000,0.000000,0.120000,0.531000,114.949500,1.876810e+05,4.000000
75%,0.806000,0.751000,8.000000,-5.235500,1.000000,0.260000,0.341000,0.000037,0.193000,0.683000,138.940000,2.128922e+05,4.000000
max,0.974000,0.990000,11.000000,-0.793000,1.000000,0.966000,0.993000,0.918000,0.973000,0.989000,216.821000,1.336000e+06,5.000000


In [15]:
# merge Top 200 data with song features

df_merge = pd.merge(df, features, left_on="ID", right_on="id", how="inner")

df_merge.to_csv("..\\data\\processed\\top200_2017_2020_w_features.csv", index=False)

df_merge.head()

,Position,Track Name,Artist,Streams,URL,Date,Start Week,End Week,ID,danceability,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,1,All I Want for Christmas Is You,Mariah Carey,1929800,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2020-12-18--2020-12-25,2020-12-18,2020-12-25,0bYg9bo50gSsH3LtXe2SQn,0.336,...,1,0.0384,0.164,0.0,0.0708,0.35,150.273,0bYg9bo50gSsH3LtXe2SQn,241107,4
1,1,All I Want for Christmas Is You,Mariah Carey,1516874,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2020-12-11--2020-12-18,2020-12-11,2020-12-18,0bYg9bo50gSsH3LtXe2SQn,0.336,...,1,0.0384,0.164,0.0,0.0708,0.35,150.273,0bYg9bo50gSsH3LtXe2SQn,241107,4
2,1,All I Want for Christmas Is You,Mariah Carey,1450763,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2020-12-04--2020-12-11,2020-12-04,2020-12-11,0bYg9bo50gSsH3LtXe2SQn,0.336,...,1,0.0384,0.164,0.0,0.0708,0.35,150.273,0bYg9bo50gSsH3LtXe2SQn,241107,4
3,7,All I Want for Christmas Is You,Mariah Carey,982738,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2020-11-27--2020-12-04,2020-11-27,2020-12-04,0bYg9bo50gSsH3LtXe2SQn,0.336,...,1,0.0384,0.164,0.0,0.0708,0.35,150.273,0bYg9bo50gSsH3LtXe2SQn,241107,4
4,23,All I Want for Christmas Is You,Mariah Carey,716897,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2020-11-20--2020-11-27,2020-11-20,2020-11-27,0bYg9bo50gSsH3LtXe2SQn,0.336,...,1,0.0384,0.164,0.0,0.0708,0.35,150.273,0bYg9bo50gSsH3LtXe2SQn,241107,4


## 3.
## Data Visualization and Modelling

To see if Dutch Spotify users actually listened to different music during the pandemic, I will focuss on 3 features.  
- Danceability: Describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
 
- Energy: A measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.  

And maybe the most important feature:
- Valence:  A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

In [16]:
# Load the data and transform date-time columns
df = pd.read_csv("..\\data\\processed\\top200_2017_2020_w_features.csv")
df[['Start Week', 'End Week']] = df[['Start Week', 'End Week']].apply(pd.to_datetime, format="%Y-%m-%d")
#df.info()

In [17]:
# calculate the weekly average on the three features
week_average = df[['Start Week', 'danceability', 'energy','valence']].groupby(pd.Grouper(key="Start Week", freq='W')).mean()

# add seperate features for the dates for easier computation
week_average["Y"] = week_average.index.year
week_average["M"] = week_average.index.month
week_average["YMD"] = week_average.index.date

week_average.head()

,danceability,energy,valence,Y,M,YMD
Start Week,,,,,,
2016-12-25,0.629000,0.641120,0.531982,2016,12,2016-12-25
2017-01-01,0.675730,0.697335,0.524167,2017,1,2017-01-01
2017-01-08,0.675095,0.678399,0.512542,2017,1,2017-01-08
2017-01-15,0.676875,0.679075,0.507017,2017,1,2017-01-15
2017-01-22,0.679265,0.675099,0.507197,2017,1,2017-01-22


In [18]:
import plotly.express as px

fig = px.line(week_average, x="YMD", y="energy", color='Y')
fig.show()

In [19]:
fig = px.line(week_average, x="YMD", y="valence", color='Y')
fig.show()

### But is this right? It seems that the number of streams in each week follows the power law, which means the first positions have much more streams than the 50th or 100th. I will, therefore, weigh the track features (valence and others) on the number of streams.

In [20]:
fig = px.line(df[df["Start Week"] >= '2020'].sort_values(by="Position"), x="Position", y="Streams", color="Start Week",
title="Distribution of streams per week in 2020")
fig.show()

### 3.2. Calculating Weighted Weekly Averages

In [66]:
# Calculating the weighted average for the most important features
weighted_week = df[['Streams', 'Start Week', 'danceability', 'energy','valence']]

for feature in ['danceability', 'energy','valence']:
    weighted_week[feature] = weighted_week[feature] * weighted_week['Streams']

weighted_week = weighted_week.groupby(["Start Week"]).sum()

for feature in ['danceability', 'energy','valence']:
    weighted_week[feature] = weighted_week[feature] / weighted_week['Streams']

# add seperate features for dates
weighted_week["Y"] = weighted_week.index.year
weighted_week["M"] = weighted_week.index.month
weighted_week["W"] = weighted_week.index.week
weighted_week["YMD"] = weighted_week.index.date

weighted_week.head()

,Streams,danceability,energy,valence,Y,M,W,YMD
Start Week,,,,,,,,
2016-12-23,48386952,0.628383,0.645799,0.545992,2016,12,51,2016-12-23
2016-12-30,45339369,0.675761,0.688582,0.531825,2016,12,52,2016-12-30
2017-01-06,52929377,0.677249,0.679035,0.538330,2017,1,1,2017-01-06
2017-01-13,55584770,0.676651,0.679837,0.530207,2017,1,2,2017-01-13
2017-01-20,54773446,0.676686,0.680728,0.527625,2017,1,3,2017-01-20


In [67]:
fig = px.line(weighted_week, x="YMD", y="valence", color='Y')
fig.show()

In [23]:
fig = px.line(weighted_week, x="W", y="valence", color='Y')
fig.show()

In [30]:
fig = px.box(weighted_week, x="Y", y="valence", color="Y")
fig.show()

### 2.3. Observations

1. There is an overall downward trend in valence over the 4 observed years.  

2. During the first COVID-19 wave (March 1 - June 1 2020), the valence is lower than in 2017 and 2018, but not very different from 2019.
3. The "Summer Peak", that starts at the end of April, is observed much later in 2020 than in other years. The 2020 peak starts exactly one week after the end of the lockdown in the Netherlands. The peak is the steepest observed.  

4. The "Holiday Peak" in the last weeks of 2020 is the same as in other years. The second/third COVID-19 wave and the second lockdown during that time did not change the valence.

Concluding: Valence (as main factor in determining music preference) did not change drasticly during the COVID-19 pandemic in 2020. Only the start of the "Summer Peak" was delayed and started just after the end of the first lockdown.

## 4. Compare with confirmed COVID-19 cases  
[CoronaWatchNL](https://github.com/J535D165/CoronaWatchNL) contains open data on COVID-19 provided by the local authorities.

In [46]:
cases = pd.read_csv("https://raw.githubusercontent.com/J535D165/CoronaWatchNL/master/data-geo/data-national/RIVM_NL_national.csv", parse_dates=["Datum"])
cases["W"] = cases["Datum"].dt.week

cases.tail(3)

,Datum,Type,Aantal,AantalCumulatief,W
921,2020-12-30,Totaal,9398.0,787300.0,53
922,2020-12-30,Ziekenhuisopname,95.0,19940.0,53
923,2020-12-30,Overleden,112.0,11324.0,53


In [42]:
fig = px.line(cases[cases["Type"]=="Totaal"], x="Datum", y="Aantal")
fig.show()

In [64]:
fig = px.line(weighted_week[(weighted_week["Y"]==2020) & (weighted_week["W"]>=cases["W"].min())], x="YMD", y="valence", color='Y')
fig.show()